In [213]:
pip install "notebook>=5.3" "ipywidgets>=7.5"

  Obtaining dependency information for notebook>=5.3 from https://files.pythonhosted.org/packages/08/e6/e032f6a4d3da85c4bd38b6ff96de51347bfb0d932ea1d2ccfd89c6374c8a/notebook-7.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for ipywidgets>=7.5 from https://files.pythonhosted.org/packages/70/1a/7edeedb1c089d63ccd8bd5c0612334774e90cf9337de9fe6c82d90081791/ipywidgets-8.1.2-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.10 from https://files.pythonhosted.org/packages/99/bc/82a8c3985209ca7c0a61b383c80e015fd92e74f8ba0ec1af98f9d6ca8dce/widgetsnbextension-4.0.10-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.10 from https://files.pythonhosted.org/packages/24/da/db1cb0387a7e4086780aff137987ee924e953d7f91b2a870f994b9b1eeb8/jupyterlab_widgets-3.0.10-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install -U kaleido

  Obtaining dependency information for kaleido from https://files.pythonhosted.org/packages/45/8e/4297556be5a07b713bb42dde0f748354de9a6918dee251c0e6bdcda341e7/kaleido-0.2.1-py2.py3-none-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 MB 7.4 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import numpy as np

In [3]:
#Read the CSV file into pandas dataframe
df = pd.read_csv('interpreter.csv', delimiter='\t')

In [4]:
#Data cleaning
"""
Delete any characters except 0-9 and the dash character:
1A -> 1
12A-14 -> 12-14
138A;138BN;140A;140B;142A;142B;144A;144B;144B1;144B2;146A;146B;148A;148B;150A;150B;152 -> 138;138;138;140;140;142;142;144;144;1441;1442;146;146;148;148;150;150;152
"""

#Remove everything except numeric characters, dash, and semicolon
df['addr:housenumber'] = [re.sub(r'[^0-9-;]', '', string) for string in df['addr:housenumber']]

#Remove dashes and semicolons at the end of the string
df['addr:housenumber'] = [re.sub(r'[-;]$', '', string) for string in df['addr:housenumber']]

"""
Take the floor of the average house number:
229-231 -> 230
1-3-5 -> 3
90-91 -> 90
"""

for i in range(len(df)):
    # Delimiter is either dash or semicolon
    numbers = re.split("-|;", df['addr:housenumber'][i])

    """
    The 3 lines of code below deal with exceptional house numbers:
    i=452472, addr:housenumber="67-E-1"
    i=454592, addr:housenumber="61-A-P"
    i=454593, addr:housenumber="136-A-G"
    i=530393, addr:housenumber="92-H - 94-H"
    """
    while '' in numbers:
        numbers.remove('')

    # Convert string data type to integer
    numbers = [int(x) for x in numbers]

    # Calculate the average if the length of numbers is greater than 1
    if len(numbers) > 1:
        """
        The 2 lines of code below handle this kind of situation: 
                @lat	@lon	addr:housenumber
        6750	52.3791941	4.8843552	58-1
        6751	52.3770529	4.8838655	58-1
        6752	52.3782579	4.8797984	58-1
        6753	52.3791942	4.8843550	58-2
        6754	52.3770530	4.8838654	58-2
        6755	52.3782579	4.8797986	58-2
        6756	52.3791942	4.8843548	58-3
        6757	52.3782579	4.8797988	58-3
        6758	52.3782579	4.8797990	58-4
        """
        while(numbers[0] > numbers[len(numbers)-1]):
            numbers.pop(len(numbers)-1)

        # Calculate the floor of average
        average = int(sum(numbers) / len(numbers))
        df.iat[i,2] = average
    else:
        # The if statement handles outlier house numbers numbers (>= 6000).
        # These outliers are results of the previous data cleaning  
        if int(numbers[0]) >= 6000:
            df.iat[i,2] = np.nan
        else:
            df.iat[i,2] = int(numbers[0])

In [5]:
df.to_csv("edited_housenumber.csv", sep='\t')

In [58]:
import plotly.graph_objects as go
import numpy as np
import plotly.io as pio

df = pd.read_csv('edited_housenumber.csv', delimiter='\t')


# df["addr:housenumber"] has median is 67.0 and mean 156.7, the distribution of house numbers is right-skewed.

# Take the log of housenumbers
df["addr:housenumber"] = np.log10(df["addr:housenumber"])

p_ticktext=[1,10,100,1000,5900]

fig = go.Figure(data=go.Scattergeo(
    lon = df["@lon"],
    lat = df["@lat"],
    mode='markers',
    marker=dict(
        color=df["addr:housenumber"]/np.log10(6000),
        colorscale="rainbow",
        size=1,
        colorbar = dict(
            thickness=60,
            titleside="top",
            title=dict(
                text="Housenumbers in Amsterdam",
                font=dict(
                    size=60
                )
            ),
            ticklen=20,
            tickwidth=5,
            ticks="outside",
            tickmode="array",
            ticktext=p_ticktext,
            tickvals=[np.log10(x)/np.log10(6000) for x in p_ticktext],
            lenmode="fraction",
            len=0.5,
            orientation="h"
        )
    ),
))

fig.update_layout(
    geo = dict(
        scope="europe",
        showland=True,
        landcolor="rgb(0, 0, 0)",
        showocean=False,
        projection = dict(
            type="mercator",
            scale=1
        ),
        resolution=50,
        lataxis = dict(
            dtick=0.05,
            range=[52.25, 52.45],
        ),
        lonaxis = dict(
            dtick=0.05,
            range=[4.70, 5.10],
        )
    )
)

pio.write_image(fig,"fruit0.png",scale=3, width=2160, height=2160)

In [303]:
# Test for data cleaning
string = "7 unit 404"
string = re.sub(r'[^0-9-;]', '', string)
string = re.sub(r'[-;]$', '', string)
numbers = re.split("-|;", string)

while '' in numbers:
    numbers.remove('')

numbers = [int(x) for x in numbers]

# Calculate the average if the length of numbers is greater than 1
if len(numbers) > 1:
    while(numbers[0] > numbers[len(numbers)-1]):
        numbers.pop(len(numbers)-1)

    # Calculate the floor of average
    average = int(sum(numbers) / len(numbers))
    df.iat[i,2] = average
else:
    if int(numbers[0]) >= 6000:
        print(numbers[0])
    df.iat[i,2] = int(numbers[0])


7404


In [54]:
np.log10(5999)*1.0/np.log10(6000)

0.9999808402466204